# netCDF plotting tool guided towards DEHM and its input data

## Info and imported packages
---

In [2]:
# Author: Zhuyun Ye, Niels S. Hvidberg
# copied to Niels 01/03 2024
# Description: Python script usefull for working with NetCDF
#              and contain tools to convert DEHM binary output 
#              to NetCDF files. The conversion requires a 
#              species file which is not present in the current
#              SPA_DEHM model.

In [3]:
# Import packages
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import datetime as dt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Opening netCDF files for plotting
---

### Functions

In [4]:
# Printing text in colors
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


In [5]:
def show(xr_dataset):
    """
    This function gives you a way to display xarray datasets per command, instead of implicitly
    by calling the dataset instance as the last thing in a cell.
    """
    from IPython.display import display, HTML
    from xarray.core.formatting_html import dataset_repr
    display(HTML(dataset_repr(xr_dataset)))

In [6]:
def xr_open_old(data_dir, days=[1, 31], hours=[0, 23]):
    infile = []
    dataset = []
    for i in range(days[0], days[1]+1):
        if (i == 1):
            z = 1
        else:
            z = 0
    
        for j in range(z+hours[0], hours[1]+1):
            name = f'201101{i:02d}{j:02d}.nc'
            infile.append(data_dir+'/'+name)
            dataset.append(xr.open_dataset(infile[-1]))
    
    return dataset

def xr_open(data_dir, days=[1, 31], hours=[0, 23]):
    infile = []
    dataset = []
    for i in range(days[0], days[1]+1):
        if (i == 1):
            z = 1
        else:
            z = 0
    
        for j in range(z+hours[0], hours[1]+1):
            name = f'1_201101{i:02d}{j:02d}.nc'
            infile.append(data_dir+'/'+name)
            dataset.append(xr.open_dataset(infile[-1]))
    
    return dataset

### Opening dataset

In [8]:
# Choose case and scroll down to the relevant section
data = 'CT'
match data:
    # DEHM data
    case 'dehm4':
        dir1 = 'netcdf1'
        dir2 = 'netcdf2'
        dir3 = 'netcdf3'
        dir4 = 'netcdf4'
        
        ds1 = xr_open_old(dir1)
        ds2 = xr_open_old(dir2)
        ds3 = xr_open_old(dir3)
        ds4 = xr_open_old(dir4)
        ds = ds1
        show(ds[0])
    
    case 'dehm test':
        ds1 = xr.open_dataset('4_2020010101.nc')
        #ds2 = xr.open_dataset('nc/jena.nc')
        ds=ds1
        show(ds)
        
    case 'dehm input':
        dir1 = 'only_cams_glob/2011/netcdf'
        dir2 = 'only_jena_ocn/2011/netcdf'
        
        ds1 = xr_open(dir1)
        ds2 = xr_open(dir1)
        ds = ds1
        show(ds[0])
    
    # CAMS-GLOB data
    case 'cams-glob':
        infile = '/home/niels/WinDoc/Data/CAMS/CAMS-GLOB/CAMS-GLOB-ANT_Glb_0.1x0.1_anthro_co2_excl_short-cycle_org_C_v5.3_monthly.nc'
        ds  = xr.open_dataset(infile)
        show(ds)
    
    case 'cams-reg':
        infile = '/home/niels/WinDoc/Data/CAMS/CAMS-REG/CAMS-REG-GHG_v6_0_emissions_year2020.nc'
        ds  = xr.open_dataset(infile)
        show(ds)
    
    # Jena data
    case 'jena': 
        infile = '/home/niels/WinDoc/Data/JENA/oc_v2023.pCO2.nc'
        ds_jena = xr.open_dataset(infile)
        ds = ds_jena
        show(ds)

    # Carbon Tracker data
    case 'CT': 
        infile1 = '/home/niels/WinDoc/Data/CT/CT-NRT.v2024-1.flux1x1.20231012.nc'
        infile2 = '/home/niels/WinDoc/Data/CT/CT-NRT.v2024-1.molefrac_components_nam1x1_2023-10-12.nc'
        infile3 = '/home/niels/WinDoc/Data/CT/CT-NRT.v2024-1.molefrac_nam1x1_2023-10-12.nc'
        ds1 = xr.open_dataset(infile1)
        ds2 = xr.open_dataset(infile2)
        ds3 = xr.open_dataset(infile3)
        ds = ds2
        show(ds)

In [19]:
ds.time.values[245]

numpy.datetime64('2020-06-01T00:00:00.000000000')

In [9]:
def get_month(year,month,first_month):
    month = (month + (12-first_month)) % 12
    year  = year - month//(11-first_month+1)

    return year, month 

get_month(2000,3,11)

(1998, 4)

## Working with dehm data to create animations
---

### Functions

In [27]:
def dehm_plotter(dataset, index=12, meandim='z', species="CO2.01", close=False, central_longitude=-32, output_dir='frames'):
    """
    Docstring for dehm_plotter:
        This function is used to plot netCDF files (output from DEHM) on a polar
        stereographic map.

        dataset: A list of xarray-dataset objects with 3D field data.
        index: The index for the specific dataset to workwith in dataset list.
        meandim: Calculate the mean over this dimension. Most often 'time' or 'z'.
        species: the kind to plot. For DEHM CO2 the tracers are called CO2.ntrac 
        (1 indexing!).

        The function creates a plot, saves it and can also close the plot again. The
        plot is saved under the name:
        
            f'{output_dir}/frame{index:03d}.png'
    """
    if meandim:
        da = dataset[index][species].mean(dim=meandim)
        lat = dataset[index]['lat'].values
        lon = dataset[index]['lon'].values
    if meandim == 0:
        da = dataset[species].isel(z=0)
        lat = dataset['lat'].values
        lon = dataset['lon'].values
    else:
        da = dataset[index][species].isel(z=0)
        lat = dataset[index]['lat'].values
        lon = dataset[index]['lon'].values
        

    
    # Get values
    pdata = da.values

    # Set min/max values for plots:
    vmin = da.min()
    vmax = da.max()*0.9
    
    # This is the map projection we want to plot *onto*
    map_proj = ccrs.NorthPolarStereo(central_longitude=central_longitude)
    fig, ax = plt.subplots(subplot_kw={'projection': map_proj}, figsize=(7,7))

    # Set coastline appearance
    ax.coastlines(color='black',zorder=2,alpha=1,linewidth=1.5)

    # Set gridlines appearance
    gl = ax.gridlines(draw_labels=True, zorder=1, alpha=0.7, linewidth=1, crs=ccrs.PlateCarree())

    # Set label style
    gl.xlabel_style = {'size':7}
    gl.ylabel_style = {'size':7}
    
    # Set colormap
    cmap = plt.get_cmap('jet')
    cmap.set_bad('w',1.0)

    # Plot image
    im = ax.pcolormesh(lon,
                       lat,
                       pdata,
                       cmap=cmap,
                       vmin=vmin, # Min value for colormap (especially used for videos)
                       vmax=vmax, # Max value for colormap
                       transform=ccrs.PlateCarree(),
                       zorder=0,  # ?
                       shading='auto')

    # Colorbar
    fig.colorbar(im, ax=ax)
    
    # Check if plot should be closed and saved instead of shown (Used for creating frames)
    if close:
        plt.savefig(f'{output_dir}/frame{index:03d}.png')
        plt.close(fig)

def make_frames_from_list(dataset, meandim, species, central_longitude=-32, output_dir='frames'):
    index = 0
    for i in range(1,32):
        if (i == 1):
            z = 1
        else:
            z = 0
    
        for j in range(z,24):
            dehm_plotter(dataset=dataset, index=index, meandim=meandim, species=species, close=True, central_longitude=central_longitude, output_dir=output_dir)
            index += 1

### Plotting single frame

In [28]:
species = f'CO2.{1:02d}'
meandim = []
central_longitude=-32

In [29]:
dehm_plotter(dataset=ds1,
             index=100,  
#            meandim=meandim, 
             meandim=0, 
             species=species, 
             close=False, 
             central_longitude=central_longitude, 
            )
plt.savefig('jena_ocean.png')

KeyError: "No variable named 'CO2.01'. Variables on the dataset include ['longitude', 'latitude', 'time', 'decimal_time', 'time_components', 'bio_flux_opt', 'ocn_flux_opt', 'fossil_flux_imp', 'fire_flux_imp']"

### Create animation

In [ ]:
if True:
    make_frames_from_list(dataset=ds1, meandim=meandim, species=species, central_longitude=central_longitude, output_dir='frames1')
    make_frames_from_list(dataset=ds2, meandim=meandim, species=species, central_longitude=central_longitude, output_dir='frames2')
    #make_frames_from_list(dataset=ds3, meandim=meandim, species=species, central_longitude=central_longitude, output_dir='frames3')
    #make_frames_from_list(dataset=ds4, meandim=meandim, species=species, central_longitude=central_longitude, output_dir='frames4')

To make a video from the frames, run this function with the desired directory as input:

> ./make_video n
> 
> ffmpeg -y -i frames\\$1/frame%03d.png -c:v libx264 -pix_fmt yuv420p video/video\\$1.mp4

n is the first argument and thus becomes $1. n is the label for the directory and output so the function reads in from frames{n} and writes to videos/video{n}.mp4.

In [ ]:
!./make_video 1
!./make_video 2
# !./make_video 3
# !./make_video 4

## Working with JENA netCDF data
---

### Functions

In [125]:
def g(y,m,d):
    """
    Returns the day number in the Gregorian calender. Check: 
    https://stackoverflow.com/a/12863278/23730517
    for explanation.
    """
    m = (m + 9) % 12
    y = y - m//10
    return 365*y + y//4 - y//100 + y//400 + (m*306 + 5)//10 + (d - 1)

def jena_day_count(y,m,d):
    return g(y,m,d) - g(1957,1,1)

In [132]:
y = 1957
m = 1
d = 1
m_org = m
y_org = y
print(f"d:{d} is kept")
print(f"m:{m} transformed to {(m + 9) % 12}")
m = (m + 9) % 12
print(f"y:{y} transformed to {y - m//10}")
y = y - m//10
print(f"days before this year:")
print(f"365*yyyy + yyyy//4 - yyyy//100 + yyyy//400 =") 
print(f"365*{y} + {y}//4 - {y}//100 + {y}//400 = {365*y + y//4 - y//100 + y//400}") 

print(f"days from this year:")
print(f"(mm*306 + 5)//10 + (dd - 1) =")
print(f"({m:2d}*306 + 5)//10 + ({d:2d} - 1) =")
print(f"(   {(m*306 + 5)//10}          +     {(d - 1)}  ) = {(m*306 + 5)//10 + (d - 1)}")

print(f"g({y:4d},{m:2d},{d:2d}) = {g(y,m,d)}")
print("g({0},{1:2d},{2:2d}) = {3}".format(y_org,m_org,d,g(1957,1,1)))

print(f"jena_day_count is: {jena_day_count(y_org,m_org,d)}")

d:1 is kept
m:1 transformed to 10
y:1957 transformed to 1956
days before this year:
365*yyyy + yyyy//4 - yyyy//100 + yyyy//400 =
365*1956 + 1956//4 - 1956//100 + 1956//400 = 714414
days from this year:
(mm*306 + 5)//10 + (dd - 1) =
(10*306 + 5)//10 + ( 1 - 1) =
(   306          +     0  ) = 306
g(1956,10, 1) = 714628
g(1957, 1, 1) = 714720
jena_day_count is: 0


In [ ]:
def JENA_plotter(dataset, species, isel={'mtime':0}, sel={}, close=False, central_longitude=-32, output_dir='plots', figspec='pCO2'):
    """
    Docstring for "JENA_plotter":
        This function is used to plot netCDF files on a polar
        stereographic map.

        dataset  : A list of xarray-dataset objects with 3D field data.
        species  : The kind to plot. For DEHM CO2 the tracers are called CO2.ntrac 
        isel/sel : A dictionary containing the variable and value or slices to choose
                   in the form {var:val/slice/index}. "isel" goes
                   by index and "sel" goes by value.
        meandim  : Calculate the mean over this dimension. Most often 'time' or 'z'.

        The function creates a plot and optionally saves and closes the plot again. The
        plot is saved under the name:
        
            f'{output_dir}/{figname}.png'
    """
    # Select values by index/slice/value.
    if isel:
        da = dataset[species].isel(isel)
    if sel:
        da = dataset[species].sel(sel)
    else:
        da = dataset[species].isel(mtime=0)

    # Set latitude and longitude values
    lat, lon = da['lat'].values, da['lon'].values
    
    # plot the first layer
    pdata = da.values
    
    # This is the map projection we want to plot *onto*
    map_proj = ccrs.NorthPolarStereo(central_longitude=central_longitude)
    fig, ax = plt.subplots(subplot_kw={'projection': map_proj}, figsize=(7,7))

    # Set appearance for coastlines
    ax.coastlines(color='black',zorder=2,alpha=1,linewidth=1.5)

    # Set appearance for gridlines
    gl = ax.gridlines(draw_labels=True, zorder=1, alpha=0.7, linewidth=1, crs=ccrs.PlateCarree())

    # Set label size
    gl.xlabel_style = {'size':7}
    gl.ylabel_style = {'size':7}

    # Set colormap
    cmap = plt.get_cmap('jet')
    cmap.set_bad('w',1.0)
    
    # Plot image
    im = ax.pcolormesh(lon,
                       lat,
                       pdata,
                       cmap=cmap,
#                      vmin=-0.1, # Min value for colormap (especially used for videos)
#                      vmax=14,   # Max value for colormap
                       transform=ccrs.PlateCarree(),
                       zorder=0,  # ?
                       shading='auto')

    # Set Colorbar
    fig.colorbar(im, ax=ax)

    # Check if plot should be closed and saved instead of shown (Used for creating frames)
    if close:
        figname = f'plot_{figspec}.png'
        plt.savefig(f'{output_dir}/{figname}.png')
        plt.close(fig)

### Plotting

In [ ]:
# Find the index for a specific time:
# 2011 = 19723
ymd = [2020, 6, 1]
sel_time = f'{ymd[0]:4d}-{ymd[1]:2d}-{ymd[2]:2d}'
sel_time_year = np.argwhere(ds.mtime.values==ds.mtime.sel(mtime=sel_time,method='nearest').values)[0]

# Example of using slice to select slices of the data. 
# This can both be done with isel (using index) or sel (using values)
_ = ds['mtime'].isel({'mtime':slice(sel_time_year[0],sel_time_year[0]+30)})
del _

In [ ]:
species = 'pCO2'
# domain 4
sel={'mtime':ds['mtime'].values[0], 'lat':slice(55.0,57.0), 'lon':slice(8.0,16.0)}
# Domain 1
sel={'mtime':ds['mtime'].values[0], 'lat':slice(20.0,90.0), 'lon':slice(-180.0,180.0)}
central_longitude=-32
# print(ds[species].sel(sel)) # Testing settings

In [ ]:
JENA_plotter(dataset=ds,
             species=species,
             sel=sel,
             central_longitude=central_longitude,
             output_dir='plots',
             figspec='pCO2',
            )

## Working with CAMS-GLOB netCDF data
---

### info

#### Key names/descriptions
*get by calling:* list(ds.data_vars)
 1. ags - Agricultural soil
 2. ene - Power generation
 3. fef - Fugitive emissions from fuels
 4. ind - Industrial processes
 5. ref - Refineries and fuel industries
 6. res - Residential
 7. shp - Ships
 8. slv - Solvents application and production
 9. sum - Sum of sectors
10. swd - Solid waste and wastewater handeling
11. tnr - Non-road transport
12. tro - Road transport

### Plotting

In [ ]:
species = 'ind'
#species = list(ds.data_vars)[5]

print('species:', species)

sel_eu = {'time':'2024-06-01', 'lat':slice(40.0,60.0), 'lon':slice(-10.0,40.0)}
sel_dk = {'time':'2024-06-01', 'lat':slice(54.0,58.0), 'lon':slice(7.0,15.5)}
sel = sel_eu

# Select values by index/slice/value.
da = ds[species].sel(sel)

# Set latitude and longitude values
lat, lon = da['lat'].values, da['lon'].values

# plot the first layer
pdata = da.values

# This is the map projection we want to plot *onto*
map_proj = ccrs.NorthPolarStereo(central_longitude=32)
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.NorthPolarStereo(central_longitude=-32)}, figsize=(15,7))
ax.coastlines(color='black',zorder=2,alpha=1,linewidth=1.5)
gl = ax.gridlines(draw_labels=True,zorder=1,alpha=0.5,linewidth=0.5,crs=ccrs.PlateCarree())
gl.xlabel_style = {'size':7}
gl.ylabel_style = {'size':7}

cmap = plt.get_cmap('jet')
cmap.set_bad('w',1.0)

im = ax.pcolormesh(lon, lat, pdata, cmap=cmap,
     vmin=0, vmax=1e-8,
    transform=ccrs.PlateCarree(),
    zorder=0, shading='auto')

plt.colorbar(im, ax=ax, orientation='vertical',label=species)
plt.show();

#### Check sum - OK  <span style="color:green">&check;</span>

In [ ]:
# Checking if the key 'sum' is equal to the actual sum.
from decimal import *

getcontext().prec = 12

ds_no_sum = ds.drop_vars('sum')
sum_sample = 0

ds_no_sum = ds_no_sum.sel(time='2023-01-01', lat=0, lon=0, method='nearest')

for key in ds_no_sum:
    sum_sample =+ ds_no_sum[key].values

print('Manual sum:', f'{sum_sample:+.8e}')
print('Automatic sum:', ds['sum'].sel(time='2023-01-01', lat=0, lon=0, method='nearest').values)
if np.isclose(sum_sample,ds['sum'].sel(time='2023-01-01', lat=0, lon=0, method='nearest').values):
    print("'sum' is correct!", bcolors.OKGREEN + u'\u2713' + bcolors.ENDC)


## Working with CAMS-REG netCDF data
---

## Working with CarbonTracker data
---

### Emission types
1. Biosphere flux
2. ocean flux
3. fossil fuel flux
4. forest fire flux

### Plotting

In [ ]:
#species = []
species = list(ds.data_vars)[1:5]

print('species:', species[:])
print('--------------------------')

sel_eu = {'latitude_source':slice(40.0,60.0), 'longitude_source':slice(-10.0,40.0)}
sel_dk = {'latitude_source':slice(54.0,58.0), 'longitude_source':slice(7.0,15.5)}
sel_area = sel_eu

sel_category = None
if False:
    sel_category = {emis_category_index}

# Select values by index/slice/value.
if sel_category:
    da = ds[species].sel(sel_area,sel_category)
else:
    da = ds[species].sel(sel_area)

# Set latitude and longitude values
lat, lon = da['lat'].values, da['lon'].values

# plot the first layer
pdata = da.values

# This is the map projection we want to plot *onto*
map_proj = ccrs.NorthPolarStereo(central_longitude=32)
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.NorthPolarStereo(central_longitude=-32)}, figsize=(15,7))
ax.coastlines(color='black',zorder=2,alpha=1,linewidth=1.5)
gl = ax.gridlines(draw_labels=True,zorder=1,alpha=0.5,linewidth=0.5,crs=ccrs.PlateCarree())
gl.xlabel_style = {'size':7}
gl.ylabel_style = {'size':7}

cmap = plt.get_cmap('jet')
cmap.set_bad('w',1.0)

im = ax.pcolormesh(lon, lat, pdata, cmap=cmap,
     vmin=0, vmax=1e-8,
    transform=ccrs.PlateCarree(),
    zorder=0, shading='auto')

plt.colorbar(im, ax=ax, orientation='vertical',label=species)
plt.show();